In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import datasets, transforms
from utils import build_dataset, flops_to_string, params_to_string, accuracy
from tqdm import tqdm_notebook

import time
from tensorboardX import SummaryWriter
writer = SummaryWriter(f'runs/MobileNet_LowRank_{int(time.time())}')
from nets.MobileNet_CIFAR_LowRank import MobileNet_CIFAR_LowRank

from thop import profile

In [3]:
class Solver(object):
    def __init__(self, model, dataset='MNIST', n_epochs=100, lr=0.001):
        self.n_epochs = n_epochs
        
        self.train_loader, self.test_loader = build_dataset(dataset, './data', batch_size = 64)
        
        self.image_dim = {'MNIST': 28*28, 'CIFAR10': 3*32*32}[dataset]
                
        self.net = model().cuda()
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
                
    def train(self):
        self.net.train()
        self.best_loss = 999
        for epoch_i in tqdm_notebook(range(self.n_epochs)):
            epoch_loss = 0
            for batch_idx, (images, labels) in enumerate(self.train_loader):
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                logits = self.net(images)
                
                loss = self.loss_fn(logits, labels)
                
                total_loss = loss

                self.optimizer.zero_grad()
                total_loss.backward()

                self.optimizer.step()
                
                epoch_loss += float(loss.data)    
            epoch_loss /= len(self.train_loader.dataset)
            # save best model
            if self.best_loss > epoch_loss:
                self.best_loss = epoch_loss
                torch.save(self.net.state_dict(), 'models/MobileNet_CIFAR_LowRank_best_loss.pth')
                
            print(f'Epoch {epoch_i} | loss: {epoch_loss:.8f}')
            
            niter = epoch_i*len(self.train_loader)+batch_idx
            writer.add_scalar('Train/Loss', loss.data[0], niter)
            self.evaluate(epoch_i)
            
        # load and test
        self.net.load_state_dict(torch.load('models/MobileNet_CIFAR_LowRank_best_loss.pth'))
        print (f'Best Loss {best_loss} | Best')
        self.evaluate(epoch_i+1)
    
    def evaluate(self, epoch_i = None):
        total = 0
        correct = 0
        self.net.eval()
        self.best_acc = 0
        for batch_idx, (images, labels) in enumerate(self.test_loader):
            images = Variable(images).cuda()

            logits = self.net(images.cuda())
            
            _, predicted = torch.max(logits.data, 1)
            
            acc1, acc5 = accuracy(logits.cpu(), labels.cpu(), topk=(1, 5))
            if self.best_acc < float(acc1):
                self.best_acc = float(acc1)
                torch.save(self.net.state_dict(), 'models/MobileNet_CIFAR_LowRank_best_acc.pth') 
            
            total += labels.size(0)
            correct += (predicted.cpu() == labels).sum()
        if epoch_i is not None:
            writer.add_scalar('Test/Acc@1', acc1, epoch_i)
            writer.add_scalar('Test/Acc@5', acc5, epoch_i)
        print(f'Top-1 Accuracy: {float(acc1)}')
        print(f'Top-5 Accuracy: {float(acc5)}')

In [4]:
standard_solver = Solver(MobileNet_CIFAR_LowRank, 'CIFAR10')

standard_solver.train()

Files already downloaded and verified
Files already downloaded and verified


Epoch 0 | loss: 0.03045103


/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Top-1 Accuracy: 25.0
Top-5 Accuracy: 93.75
Epoch 1 | loss: 0.02639024
Top-1 Accuracy: 25.0
Top-5 Accuracy: 100.0
Epoch 2 | loss: 0.02420982
Top-1 Accuracy: 56.25
Top-5 Accuracy: 100.0
Epoch 3 | loss: 0.02276857
Top-1 Accuracy: 43.75
Top-5 Accuracy: 87.5
Epoch 4 | loss: 0.02162330
Top-1 Accuracy: 56.25
Top-5 Accuracy: 100.0


KeyboardInterrupt: 

In [ ]:
nn.BatchNorm2d?